# Assignment 8: Data Aggregation and Group Operations

## Overview
This assignment covers data aggregation and group operations using healthcare/insurance EHR data. You'll analyze patient encounters (insurance claims), healthcare providers, and medical facilities across different regions. The data includes provider statistics, facility information, and encounter charges with insurance coverage details.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
os.makedirs('output', exist_ok=True)

## Question 1: Basic GroupBy Operations

### Part 1.1: Load and Explore Data

**TODO: Load and merge the datasets**

In [3]:
# TODO: Load the three CSV files into dataframes

# facility_df = pd.read_csv(...)
# encounter_df = pd.read_csv(...)

# TODO: Print the shapes of each dataframe
# print("Provider data shape:", ...)
# print("Facility data shape:", ...)
# print("Encounter data shape:", ...)

# TODO: Merge the dataframes for analysis
# Hint: Start by merging encounter_df with provider_df on 'provider_id'
#       Then merge the result with facility_df on 'facility_id'
# Note: Both encounter_df and provider_df have 'facility_id', and both encounter_df and facility_df have 'region'
#       After merging, you may need to handle duplicate column names (pandas will add suffixes like _x and _y)
#       You can drop one of the duplicate columns and rename if needed

# TODO: Print the merged dataframe shape, columns, and first few rows
# print("\nMerged data shape:", ...)
# print("\nColumns:", ...)
# print("\nFirst few rows:")
# print(...)
import pandas as pd

# TODO: Load the three CSV files into dataframes
provider_df = pd.read_csv("data/provider_data.csv")
facility_df = pd.read_csv("data/facility_data.csv")
encounter_df = pd.read_csv("data/encounter_data.csv")

# TODO: Print the shapes of each dataframe
print("Provider data shape:", provider_df.shape)
print("Facility data shape:", facility_df.shape)
print("Encounter data shape:", encounter_df.shape)

# TODO: Merge the dataframes for analysis
# Step 1: merge encounter with provider on provider_id
merged_df = encounter_df.merge(
    provider_df,
    on="provider_id",
    how="left",
    suffixes=("", "_provider")
)

# Step 2: merge with facility on facility_id
merged_df = merged_df.merge(
    facility_df,
    on="facility_id",
    how="left",
    suffixes=("", "_facility")
)

# Handle duplicate region columns if present
if "region_provider" in merged_df.columns:
    merged_df = merged_df.drop(columns=["region_provider"])

if "region_facility" in merged_df.columns:
    merged_df = merged_df.drop(columns=["region_facility"])

# TODO: Print the merged dataframe shape, columns, and first few rows
print("\nMerged data shape:", merged_df.shape)
print("\nColumns:")
print(merged_df.columns.tolist())

print("\nFirst few rows:")
print(merged_df.head())


Provider data shape: (500, 7)
Facility data shape: (10, 6)
Encounter data shape: (5000, 12)

Merged data shape: (5000, 22)

Columns:
['encounter_id', 'patient_id', 'provider_id', 'facility_id', 'encounter_date', 'encounter_type', 'diagnosis_code', 'procedure_code', 'service_charge', 'insurance_paid', 'patient_paid', 'region', 'provider_name', 'provider_type', 'facility_id_provider', 'specialty', 'years_experience', 'license_number', 'facility_name', 'facility_type', 'beds', 'established_date']

First few rows:
  encounter_id patient_id provider_id facility_id encounter_date  \
0     ENC00001   PAT01564      PR0259      FAC004     2023-08-21   
1     ENC00002   PAT00296      PR0258      FAC001     2023-09-11   
2     ENC00003   PAT01381      PR0213      FAC001     2023-03-20   
3     ENC00004   PAT01131      PR0146      FAC004     2023-09-03   
4     ENC00005   PAT01388      PR0354      FAC003     2023-06-14   

         encounter_type diagnosis_code  procedure_code  service_charge  \
0

### Part 1.2: Basic Aggregation

**TODO: Perform basic groupby operations**

In [4]:
# TODO: Group by facility (healthcare facility) and calculate basic stats
# TODO: Calculate mean, sum, count for years_experience by facility
# TODO: Calculate total encounter charges by facility
# TODO: Calculate total insurance payments by facility
# TODO: Find the facility with highest total encounter charges

# TODO: Create a summary dataframe with one row per facility showing:
#       facility_name, total_providers, avg_years_experience, total_service_charge, total_insurance_paid
# TODO: Save the summary dataframe as 'output/q1_groupby_analysis.csv'

# Group by facility and calculate required statistics

facility_summary = (
    merged_df
    .groupby(["facility_id", "facility_name"])
    .agg(
        total_providers=("provider_id", "nunique"),
        avg_years_experience=("years_experience", "mean"),
        total_service_charge=("service_charge", "sum"),
        total_insurance_paid=("insurance_paid", "sum"),
        encounter_count=("encounter_id", "count")
    )
    .reset_index()
)

# Find the facility with the highest total encounter charges
highest_charge_facility = facility_summary.loc[
    facility_summary["total_service_charge"].idxmax()
]

print("Facility with highest total encounter charges:")
print(highest_charge_facility)

# Select only required columns for output
facility_summary = facility_summary[
    [
        "facility_name",
        "total_providers",
        "avg_years_experience",
        "total_service_charge",
        "total_insurance_paid",
    ]
]

# Save output
facility_summary.to_csv("output/q1_groupby_analysis.csv", index=False)

facility_summary.head()


Facility with highest total encounter charges:
facility_id                            FAC007
facility_name           Pediatric Care Center
total_providers                           326
avg_years_experience                19.811676
total_service_charge                361811.04
total_insurance_paid                291790.95
encounter_count                           531
Name: 6, dtype: object


,facility_name,total_providers,avg_years_experience,total_service_charge,total_insurance_paid
0,City General Hospital,312,19.563008,295508.13,236798.50
1,Community Health Clinic,328,20.160784,334992.21,269207.92
2,Regional Medical Center,309,20.209205,316284.98,251750.43
3,Northside Urgent Care,305,20.137014,297283.06,236839.71
4,Cardiology Specialty Center,297,19.319728,276658.83,222139.04


In [13]:
with open("output/q1_aggregation_report.txt", "w") as f:
    f.write("Q1 Aggregation Report\n")
    f.write("=====================\n\n")

    f.write("Question 1: GroupBy aggregation and transform operations\n")
    f.write("----------------------------------------------------------\n")
    f.write("This report summarizes facility-level aggregations created using\n")
    f.write("pandas groupby().transform() operations.\n\n")

    f.write("Columns added:\n")
    f.write("- facility_mean_years_experience (mean years_experience per facility)\n")
    f.write("- facility_std_years_experience (std dev of years_experience per facility)\n")
    f.write("- years_experience_zscore (z-score within facility)\n")
    f.write("- facility_total_service_charge (sum of service_charge per facility)\n\n")

    f.write("Summary statistics for years_experience:\n")
    f.write("----------------------------------------\n")
    f.write(merged_df["years_experience"].describe().to_string())
    f.write("\n\n")

    f.write("Summary statistics for facility_mean_years_experience:\n")
    f.write("-------------------------------------------------------\n")
    f.write(merged_df["facility_mean_years_experience"].describe().to_string())
    f.write("\n\n")

    f.write("Sample transformed rows:\n")
    f.write("-------------------------\n")
    f.write(
        merged_df[
            [
                "facility_id",
                "years_experience",
                "facility_mean_years_experience",
                "facility_std_years_experience",
                "years_experience_zscore",
                "facility_total_service_charge",
            ]
        ]
        .head(5)
        .to_string(index=False)
    )


### Part 1.3: Transform Operations

**TODO: Use transform operations to add group statistics**

In [5]:
# TODO: Add facility mean years_experience as new column
# TODO: Add facility standard deviation of years_experience
# TODO: Create normalized years_experience (z-score within facility)
# TODO: Add facility total encounter charges as new column

# TODO: Display the enhanced dataframe
# TODO: Save results as 'output/q1_aggregation_report.txt'
# Add facility-level statistics using transform

# Mean years of experience per facility
merged_df["facility_mean_years_experience"] = (
    merged_df
    .groupby("facility_id")["years_experience"]
    .transform("mean")
)

# Standard deviation of years of experience per facility
merged_df["facility_std_years_experience"] = (
    merged_df
    .groupby("facility_id")["years_experience"]
    .transform("std")
)

# Normalized years_experience (z-score within facility)
merged_df["years_experience_zscore"] = (
    (merged_df["years_experience"] - merged_df["facility_mean_years_experience"]) /
    merged_df["facility_std_years_experience"]
)

# Total encounter charges per facility
merged_df["facility_total_service_charge"] = (
    merged_df
    .groupby("facility_id")["service_charge"]
    .transform("sum")
)

# Display the enhanced dataframe
print(merged_df.head())

# Save aggregation report
with open("output/q1_aggregation_report.txt", "w") as f:
    f.write("Q1 Aggregation Report\n")
    f.write("=====================\n\n")
    f.write("Added the following facility-level columns using groupby().transform():\n")
    f.write("- facility_mean_years_experience\n")
    f.write("- facility_std_years_experience\n")
    f.write("- years_experience_zscore\n")
    f.write("- facility_total_service_charge\n")


  encounter_id patient_id provider_id facility_id encounter_date  \
0     ENC00001   PAT01564      PR0259      FAC004     2023-08-21   
1     ENC00002   PAT00296      PR0258      FAC001     2023-09-11   
2     ENC00003   PAT01381      PR0213      FAC001     2023-03-20   
3     ENC00004   PAT01131      PR0146      FAC004     2023-09-03   
4     ENC00005   PAT01388      PR0354      FAC003     2023-06-14   

         encounter_type diagnosis_code  procedure_code  service_charge  \
0          Office Visit          M79.3           99213          345.03   
1             Inpatient          N18.9           85025          550.48   
2                   Lab          K21.9           80053          149.84   
3  Outpatient Procedure          H52.9           70450         1468.48   
4             Emergency          G43.9           99214          672.23   

   insurance_paid  ...  years_experience license_number  \
0          290.33  ...                11      LIC000259   
1          482.51  ...      

## Question 2: Advanced GroupBy Operations

### Part 2.1: Filter Operations

**TODO: Use filter operations to remove groups**

In [6]:
# TODO: Filter facilities with more than 20 providers (sufficient staffing)
# TODO: Filter facilities with average years_experience > 10 (experienced staff)
# TODO: Filter facilities with total encounter charges > 250000 (high volume)

# TODO: Create a summary of filtered results
# TODO: Save results as 'output/q2_filter_analysis.csv'

# Q2: Advanced GroupBy – filter operations

# Apply successive groupby filters on facility_id
filtered_df = (
    merged_df
    # Facilities with more than 20 unique providers
    .groupby("facility_id")
    .filter(lambda x: x["provider_id"].nunique() > 20)
    # Facilities with average years_experience > 10
    .groupby("facility_id")
    .filter(lambda x: x["years_experience"].mean() > 10)
    # Facilities with total encounter charges > 250000
    .groupby("facility_id")
    .filter(lambda x: x["service_charge"].sum() > 250000)
)

# Create a summary of filtered facilities
filtered_summary = (
    filtered_df
    .groupby(["facility_id", "facility_name"])
    .agg(
        total_providers=("provider_id", "nunique"),
        avg_years_experience=("years_experience", "mean"),
        total_service_charge=("service_charge", "sum"),
        total_insurance_paid=("insurance_paid", "sum"),
        encounter_count=("encounter_id", "count")
    )
    .reset_index()
)

# Save results
filtered_summary.to_csv("output/q2_filter_analysis.csv", index=False)

# Display summary
filtered_summary


,facility_id,facility_name,total_providers,avg_years_experience,total_service_charge,total_insurance_paid,encounter_count
0,FAC001,City General Hospital,312,19.563008,295508.13,236798.50,492
1,FAC002,Community Health Clinic,328,20.160784,334992.21,269207.92,510
2,FAC003,Regional Medical Center,309,20.209205,316284.98,251750.43,478
3,FAC004,Northside Urgent Care,305,20.137014,297283.06,236839.71,489
4,FAC005,Cardiology Specialty Center,297,19.319728,276658.83,222139.04,441
5,FAC006,Oncology Treatment Center,324,19.381139,334927.30,267183.95,509
6,FAC007,Pediatric Care Center,326,19.811676,361811.04,291790.95,531
7,FAC008,Emergency Medical Center,299,19.668687,310735.25,248439.20,495
8,FAC009,Primary Care Clinic,316,20.102970,323315.71,260297.44,505
9,FAC010,Surgical Specialty Center,340,20.778182,349909.18,278503.38,550


### Part 2.2: Apply Operations

**TODO: Use apply operations with custom functions**

In [7]:
# TODO: Create custom function to calculate provider statistics
def provider_stats(group):
    # TODO: Return mean, std, min, max, range for years_experience
    pass

# TODO: Apply custom function to each facility
# Note: Add include_groups=False to avoid pandas FutureWarning about groupby columns
facility_provider_stats = merged_df.groupby('facility_name').apply(provider_stats, include_groups=False)

# TODO: Create function to find top providers in each facility
def top_providers(group, n=2):
    """Return top n providers in a group by encounter volume"""
    # TODO: Return top n rows by encounter count, selecting only provider_id, provider_name, 
    #       years_experience, specialty
    pass

# TODO: Apply function to get top 2 providers per facility
# IMPORTANT: When using groupby().apply(), the result has a MultiIndex where:
# - Level 0: The grouping column (facility_name)
# - Level 1: The original row index from each group
# Use reset_index(level=0) to move only the grouping column back to a regular column
# DO NOT use reset_index(drop=True) - that would drop ALL index levels including facility_name!
top_providers_by_facility = merged_df.groupby('facility_name').apply(
    lambda x: top_providers(x, n=2)
).reset_index(level=0)  # Moves facility_name from index to column

# TODO: Save results as 'output/q2_performance_report.txt'

# Q2: Advanced GroupBy – apply operations

# Custom function to calculate provider statistics
def provider_stats(group):
    return pd.Series({
        "mean_years_experience": group["years_experience"].mean(),
        "std_years_experience": group["years_experience"].std(),
        "min_years_experience": group["years_experience"].min(),
        "max_years_experience": group["years_experience"].max(),
        "range_years_experience": group["years_experience"].max() - group["years_experience"].min()
    })

# Apply custom function to each facility
facility_provider_stats = (
    merged_df
    .groupby("facility_name")
    .apply(provider_stats, include_groups=False)
)

print("Facility-level provider statistics:")
print(facility_provider_stats.head())

# Function to find top providers in each facility by encounter volume
def top_providers(group, n=2):
    provider_counts = (
        group
        .groupby(
            ["provider_id", "provider_name", "years_experience", "specialty"]
        )
        .size()
        .reset_index(name="encounter_count")
        .sort_values("encounter_count", ascending=False)
        .head(n)
    )
    return provider_counts[
        ["provider_id", "provider_name", "years_experience", "specialty", "encounter_count"]
    ]

# Apply function to get top 2 providers per facility
top_providers_by_facility = (
    merged_df
    .groupby("facility_name")
    .apply(lambda x: top_providers(x, n=2))
    .reset_index(level=0)  # keep facility_name
)

print("\nTop providers by facility:")
print(top_providers_by_facility.head())

# Save performance report
with open("output/q2_performance_report.txt", "w") as f:
    f.write("Q2 Provider Performance Report\n")
    f.write("==============================\n\n")

    f.write("Facility-level provider experience statistics:\n")
    f.write(facility_provider_stats.to_string())
    f.write("\n\n")

    f.write("Top 2 providers per facility by encounter volume:\n")
    f.write(top_providers_by_facility.to_string(index=False))


Facility-level provider statistics:
                             mean_years_experience  std_years_experience  \
facility_name                                                              
Cardiology Specialty Center              19.319728             10.484612   
City General Hospital                    19.563008             10.702137   
Community Health Clinic                  20.160784             11.044011   
Emergency Medical Center                 19.668687             11.023867   
Northside Urgent Care                    20.137014             10.581439   

                             min_years_experience  max_years_experience  \
facility_name                                                             
Cardiology Specialty Center                   1.0                  39.0   
City General Hospital                         1.0                  39.0   
Community Health Clinic                       1.0                  39.0   
Emergency Medical Center                      1.0       

/tmp/ipykernel_8902/1104135206.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_providers_by_facility = merged_df.groupby('facility_name').apply(
/tmp/ipykernel_8902/1104135206.py:71: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: top_providers(x, n=2))


### Part 2.3: Hierarchical Grouping

**TODO: Perform multi-level grouping**

In [8]:
# TODO: Group by facility type and region
# TODO: Calculate statistics for each facility_type-region combination
# TODO: Use unstack to convert to wide format
# TODO: Use stack to convert back to long format

# TODO: Analyze the hierarchical structure
# TODO: Save results as 'output/q2_hierarchical_analysis.csv'

# Q2: Hierarchical grouping with MultiIndex

# Group by facility_type and region and calculate statistics
hierarchical_stats = (
    merged_df
    .groupby(["facility_type", "region"])
    .agg(
        total_encounters=("encounter_id", "count"),
        total_service_charge=("service_charge", "sum"),
        avg_service_charge=("service_charge", "mean"),
        avg_years_experience=("years_experience", "mean")
    )
)

print("Hierarchical (MultiIndex) result:")
print(hierarchical_stats.head())

# Convert to wide format using unstack (region becomes columns)
wide_stats = hierarchical_stats.unstack("region")

print("\nWide-format (after unstack):")
print(wide_stats.head())

# Convert back to long format using stack
long_stats = wide_stats.stack("region")

print("\nBack to long format (after stack):")
print(long_stats.head())

# Reset index for clean output
hierarchical_analysis = long_stats.reset_index()

# Save results
hierarchical_analysis.to_csv(
    "output/q2_hierarchical_analysis.csv",
    index=False
)

hierarchical_analysis.head()


Hierarchical (MultiIndex) result:
                         total_encounters  total_service_charge  \
facility_type    region                                           
Hospital         South                509             334927.30   
Specialty Center South               1026             672546.29   
                 West                 946             599974.54   
Urgent Care      North                510             334992.21   
                 South                970             611793.11   

                         avg_service_charge  avg_years_experience  
facility_type    region                                            
Hospital         South           658.010413             19.381139  
Specialty Center South           655.503207             19.742690  
                 West            634.222558             19.737844  
Urgent Care      North           656.847471             20.160784  
                 South           630.714546             19.881443  

Wide-format (after 

/tmp/ipykernel_8902/4069786285.py:33: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  long_stats = wide_stats.stack("region")


,facility_type,region,total_encounters,total_service_charge,avg_service_charge,avg_years_experience
0,Hospital,South,509.0,334927.30,658.010413,19.381139
1,Specialty Center,South,1026.0,672546.29,655.503207,19.742690
2,Specialty Center,West,946.0,599974.54,634.222558,19.737844
3,Urgent Care,North,510.0,334992.21,656.847471,20.160784
4,Urgent Care,South,970.0,611793.11,630.714546,19.881443


In [14]:
with open("output/q2_performance_report.txt", "w") as f:
    f.write("Q2 Provider Performance Report\n")
    f.write("==============================\n\n")

    f.write("Question 2: Advanced GroupBy operations using apply()\n")
    f.write("------------------------------------------------------\n")
    f.write("This report summarizes provider-level performance metrics\n")
    f.write("calculated using pandas groupby().apply() operations.\n\n")

    f.write("Facility-level provider experience statistics:\n")
    f.write("------------------------------------------------\n")
    f.write("Statistics computed: mean, std, min, max, range of years_experience\n\n")

    f.write(facility_provider_stats.to_string())
    f.write("\n\n")

    f.write("Top providers per facility by encounter volume:\n")
    f.write("------------------------------------------------\n")
    f.write(
        top_providers_by_facility[
            [
                "facility_name",
                "provider_id",
                "provider_name",
                "years_experience",
                "specialty",
                "encounter_count",
            ]
        ].to_string(index=False)
    )


## Question 3: Pivot Tables and Cross-Tabulations

### Part 3.1: Basic Pivot Tables

**TODO: Create pivot tables for multi-dimensional analysis**

In [15]:
# TODO: Create pivot table: encounter charges by procedure code and region
# TODO: Create pivot table with multiple aggregations (sum, mean, count)
# TODO: Add totals (margins) to pivot table
# TODO: Handle missing values with fill_value

# Tip: When saving a pivot table to CSV, the index (row labels) is preserved by default.
#      This is usually desired - when reading back, use index_col=0 to restore the index.
# TODO: Save results as 'output/q3_pivot_analysis.csv'

# Q3: Pivot tables – encounter charges by procedure code and region

pivot_table = pd.pivot_table(
    merged_df,
    values="service_charge",
    index="procedure_code",
    columns="region",
    aggfunc=["sum", "mean", "count"],
    margins=True,
    margins_name="Total",
    fill_value=0
)

# Flatten MultiIndex columns so CSV has numeric columns
pivot_table_flat = pivot_table.copy()
pivot_table_flat.columns = [
    f"{agg}_{region}" for agg, region in pivot_table_flat.columns
]

# Save flattened pivot table
pivot_table_flat.to_csv("output/q3_pivot_analysis.csv")


### Part 3.2: Cross-Tabulations

**TODO: Create cross-tabulations for categorical analysis**

In [10]:
# TODO: Create crosstab of facility type vs region with counts
# TODO: Create crosstab with margins (row and column totals)
# TODO: Create multi-dimensional crosstab showing facility type × encounter type × region combinations
#       (Hint: use pd.crosstab with multiple index/column parameters, or create manually from the data)

# TODO: Analyze the cross-tabulation results
# TODO: Save results as 'output/q3_crosstab_analysis.csv'
# Q3: Cross-tabulations

# Crosstab of facility type vs region (counts)
facility_region_crosstab = pd.crosstab(
    merged_df["facility_type"],
    merged_df["region"]
)

print("Facility type vs region crosstab:")
print(facility_region_crosstab)

# Crosstab with margins (row and column totals)
facility_region_crosstab_margins = pd.crosstab(
    merged_df["facility_type"],
    merged_df["region"],
    margins=True,
    margins_name="Total"
)

print("\nFacility type vs region crosstab with totals:")
print(facility_region_crosstab_margins)

# Multi-dimensional crosstab:
# Facility type × Encounter type × Region
multi_crosstab = pd.crosstab(
    index=[merged_df["facility_type"], merged_df["encounter_type"]],
    columns=merged_df["region"]
)

print("\nMulti-dimensional crosstab:")
print(multi_crosstab.head())

# Save results (save the most comprehensive version)
facility_region_crosstab_margins.to_csv(
    "output/q3_crosstab_analysis.csv"
)


Facility type vs region crosstab:
region            North  South  West
facility_type                       
Hospital              0    509     0
Specialty Center      0   1026   946
Urgent Care         510    970  1039

Facility type vs region crosstab with totals:
region            North  South  West  Total
facility_type                              
Hospital              0    509     0    509
Specialty Center      0   1026   946   1972
Urgent Care         510    970  1039   2519
Total               510   2505  1985   5000

Multi-dimensional crosstab:
region                              North  South  West
facility_type encounter_type                          
Hospital      Emergency                 0    127     0
              Inpatient                 0     95     0
              Lab                       0    100     0
              Office Visit              0     85     0
              Outpatient Procedure      0    102     0


### Part 3.3: Pivot Table Visualization

**TODO: Create visualizations from pivot tables**

In [11]:
# TODO: Create heatmap from pivot table
# TODO: Create bar chart from pivot table
# TODO: Customize colors and styling
# TODO: Add appropriate titles and labels

# TODO: Save the plot as 'output/q3_pivot_visualization.png'

import matplotlib.pyplot as plt
import numpy as np

# Use the SUM layer of the pivot table (exclude Total row/column for plotting)
pivot_sum = pivot_table["sum"].drop(index="Total", errors="ignore")
pivot_sum = pivot_sum.drop(columns="Total", errors="ignore")

# Create figure with two plots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# -----------------------------
# Heatmap: Service charges
# -----------------------------
im = axes[0].imshow(pivot_sum.values, aspect="auto")

axes[0].set_title("Total Service Charges by Procedure and Region")
axes[0].set_xlabel("Region")
axes[0].set_ylabel("Procedure Code")

axes[0].set_xticks(np.arange(len(pivot_sum.columns)))
axes[0].set_xticklabels(pivot_sum.columns, rotation=45, ha="right")

axes[0].set_yticks(np.arange(len(pivot_sum.index)))
axes[0].set_yticklabels(pivot_sum.index)

plt.colorbar(im, ax=axes[0], label="Total Service Charge")

# -----------------------------
# Bar chart: Total charges by region
# -----------------------------
region_totals = pivot_sum.sum(axis=0)

axes[1].bar(region_totals.index, region_totals.values)
axes[1].set_title("Total Service Charges by Region")
axes[1].set_xlabel("Region")
axes[1].set_ylabel("Total Service Charge")

# Layout and save
plt.tight_layout()
plt.savefig("output/q3_pivot_visualization.png", dpi=300)
plt.close()


## Submission Checklist

Before submitting, verify you've created:

- [ ] `output/q1_groupby_analysis.csv` - Basic groupby analysis
- [ ] `output/q1_aggregation_report.txt` - Aggregation report
- [ ] `output/q2_filter_analysis.csv` - Filter operations analysis
- [ ] `output/q2_hierarchical_analysis.csv` - Hierarchical analysis
- [ ] `output/q2_performance_report.txt` - Performance report
- [ ] `output/q3_pivot_analysis.csv` - Pivot table analysis
- [ ] `output/q3_crosstab_analysis.csv` - Cross-tabulation analysis
- [ ] `output/q3_pivot_visualization.png` - Pivot visualization

## Key Learning Objectives

- Master the split-apply-combine paradigm
- Apply aggregation functions and transformations
- Create pivot tables for multi-dimensional analysis
- Apply advanced groupby techniques